In [1]:
import sys
sys.path.append('../modules')
from lonlat_to_healpix import *

%load_ext autoreload
%autoreload 2

In [2]:
# Create a small test DataArray with lon/lat coordinates
import numpy as np
import xarray as xr

# Create a small 3x3 grid around Biscay location
n_rows, n_cols = 3, 3
center_lon, center_lat = -4.5, 48.0  # Biscay coordinates

# Create a regular grid with 
lon_spacing = 0.01  # degrees
lat_spacing = 0.01  # degrees

lon = np.linspace(center_lon - (n_cols-1)*lon_spacing/2, 
                  center_lon + (n_cols-1)*lon_spacing/2, n_cols)
lat = np.linspace(center_lat - (n_rows-1)*lat_spacing/2, 
                  center_lat + (n_rows-1)*lat_spacing/2, n_rows)

lon_2d, lat_2d = np.meshgrid(lon, lat)


# Create some dummy data
test_data = np.round(np.random.rand(n_rows, n_cols), 4)

# Create DataArray with lon/lat coordinates
test_da = xr.DataArray(
    test_data,
    dims=['rows', 'columns'],
    coords={
        'longitude': (['rows', 'columns'], lon_2d),
        'latitude': (['rows', 'columns'], lat_2d),
    },
    name='test_variable'
)

print("Created test DataArray:")
print(f"  Shape: {test_da.shape}")
print(f"  Lon range: {lon_2d.min():.4f} to {lon_2d.max():.4f}")
print(f"  Lat range: {lat_2d.min():.4f} to {lat_2d.max():.4f}")
test_da


Created test DataArray:
  Shape: (3, 3)
  Lon range: -4.5100 to -4.4900
  Lat range: 47.9900 to 48.0100


<xarray.DataArray 'test_variable' (rows: 3, columns: 3)> Size: 72B
array([[0.4927, 0.1547, 0.2769],
       [0.801 , 0.3462, 0.2815],
       [0.4343, 0.605 , 0.7028]])
Coordinates:
    longitude  (rows, columns) float64 72B -4.51 -4.5 -4.49 ... -4.51 -4.5 -4.49
    latitude   (rows, columns) float64 72B 47.99 47.99 47.99 ... 48.01 48.01
Dimensions without coordinates: rows, columns

In [3]:
test_da_with_healpix = add_healpix_to_dataarray(test_da, verbose=True)

Calculating pixel sizes...
PIXEL SIZE STATISTICS

Row-wise pixel sizes (North-South direction):
  Mean:   1111.90 m
  Median: 1111.90 m
  Min:    1111.90 m
  Max:    1111.90 m

Column-wise pixel sizes (East-West direction):
  Mean:   746.25 m
  Median: 746.25 m
  Min:    746.11 m
  Max:    746.40 m

Using pixel size: 1111.90 m (max of row/column)

Selected HEALPix level: 12
  Pixel size: 1111.90 m
  Edge length: 1591.71 m
  Ratio: 1.43x

Calculating HEALPix cell IDs at level 12...
Added HEALPix coordinates:
  Valid pixels: 9 / 9
  Unique cells: 5
  Level: 12
  Grid mapping: crs (CF-compliant)


/home/kajetan/miniconda3/envs/healpix/lib/python3.13/site-packages/numpy/_core/numeric.py:387: RuntimeWarning: invalid value encountered in cast
  multiarray.copyto(a, fill_value, casting='unsafe')


In [4]:
# Inspect the result
print("\nTest DataArray with HEALPix coordinates:")
print(f"  Coordinates: {list(test_da_with_healpix.coords.keys())}")

# Get HEALPix level from crs coordinate (CF convention)
if 'crs' in test_da_with_healpix.coords:
    crs_attrs = test_da_with_healpix.coords['crs'].attrs
    healpix_level = crs_attrs.get('refinement_level', 'unknown')
    indexing_scheme = crs_attrs.get('indexing_scheme', 'unknown')
    reference_body = crs_attrs.get('reference_body', 'unknown')
    print(f"  HEALPix level: {healpix_level}")
    print(f"  Indexing scheme: {indexing_scheme}")
    print(f"  Reference body: {reference_body}")
else:
    print(f"  HEALPix level: unknown (crs coordinate not found)")

print(f"\nHEALPix cell IDs shape: {test_da_with_healpix.coords['healpix'].shape}")
print(f"Unique HEALPix cells: {len(np.unique(test_da_with_healpix.coords['healpix'].values[~np.isnan(test_da_with_healpix.coords['healpix'].values)]))}")
print(f"\nHEALPix cell IDs:")
print(test_da_with_healpix.coords['healpix'].values)

# Show CRS metadata
if 'crs' in test_da_with_healpix.coords:
    print(f"\nCRS metadata:")
    for key, value in test_da_with_healpix.coords['crs'].attrs.items():
        print(f"  {key}: {value}")



Test DataArray with HEALPix coordinates:
  Coordinates: ['longitude', 'latitude', 'healpix', 'crs']
  HEALPix level: 12
  Indexing scheme: nested
  Reference body: WGS84

HEALPix cell IDs shape: (3, 3)
Unique HEALPix cells: 5

HEALPix cell IDs:
[[56464103 56464103 56464114]
 [56464109 56464109 56464114]
 [56464111 56464111 56464120]]

CRS metadata:
  grid_mapping_name: healpix
  refinement_level: 12
  indexing_scheme: nested
  reference_body: WGS84
  refinement_ratio: 4
